In [1]:
import os
from google.cloud import storage

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../t5/prepare/mesolitica-tpu.json'
client = storage.Client()
bucket = client.bucket('mesolitica-tpu-general')
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from t5.data import preprocessors as prep
import functools
import t5
import gin
import sentencepiece as spm
from glob import glob
import os
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry

gin.parse_config_file('pretrained_models_base_operative_config.gin')
vocab = 'sp10m.cased.t5.model'
sp = spm.SentencePieceProcessor()
sp.Load(vocab)

True

In [3]:
# import sentencepiece as spm
# vocab = 'sp10m.cased.t5.model'
# sp = spm.SentencePieceProcessor()
# sp.Load(vocab)

In [4]:
def cnn_dataset(split, shuffle_files = False):
    del shuffle_files
    ds = tf.data.TextLineDataset(glob('t5-data/cnn-summarization-*.tsv'))

    ds = ds.map(
        functools.partial(
            tf.io.decode_csv,
            record_defaults = ['', ''],
            field_delim = '\t',
            use_quote_delim = False,
        ),
        num_parallel_calls = tf.data.experimental.AUTOTUNE,
    )
    ds = ds.map(lambda *ex: dict(zip(['question', 'answer'], ex)))
    return ds


def cnn_preprocessor(ds):
    def to_inputs_and_targets(ex):
        return {
            'inputs': tf.strings.join(['ringkasan: ', ex['question']]),
            'targets': ex['answer'],
        }

    return ds.map(
        to_inputs_and_targets,
        num_parallel_calls = tf.data.experimental.AUTOTUNE,
    )


t5.data.TaskRegistry.remove('cnn_dataset')
t5.data.TaskRegistry.add(
    'cnn_dataset',
    dataset_fn = cnn_dataset,
    splits = ['train'],
    text_preprocessor = [cnn_preprocessor],
    sentencepiece_model_path = vocab,
    metric_fns = [t5.evaluation.metrics.accuracy],
)


def multinews_dataset(split, shuffle_files = False):
    del shuffle_files
    ds = tf.data.TextLineDataset(glob('t5-data/multinews-summarization-*.tsv'))

    ds = ds.map(
        functools.partial(
            tf.io.decode_csv,
            record_defaults = ['', ''],
            field_delim = '\t',
            use_quote_delim = False,
        ),
        num_parallel_calls = tf.data.experimental.AUTOTUNE,
    )
    ds = ds.map(lambda *ex: dict(zip(['question', 'answer'], ex)))
    return ds


def multinews_preprocessor(ds):
    def to_inputs_and_targets(ex):
        return {
            'inputs': tf.strings.join(['ringkasan: ', ex['question']]),
            'targets': ex['answer'],
        }

    return ds.map(
        to_inputs_and_targets,
        num_parallel_calls = tf.data.experimental.AUTOTUNE,
    )


t5.data.TaskRegistry.remove('multinews_dataset')
t5.data.TaskRegistry.add(
    'multinews_dataset',
    dataset_fn = multinews_dataset,
    splits = ['train'],
    text_preprocessor = [multinews_preprocessor],
    sentencepiece_model_path = vocab,
    metric_fns = [t5.evaluation.metrics.accuracy],
)

def news_dataset(split, shuffle_files = False):
    del shuffle_files
    ds = tf.data.TextLineDataset(glob('t5-data/news-title-*.tsv'))

    ds = ds.map(
        functools.partial(
            tf.io.decode_csv,
            record_defaults = ['', ''],
            field_delim = '\t',
            use_quote_delim = False,
        ),
        num_parallel_calls = tf.data.experimental.AUTOTUNE,
    )
    ds = ds.map(lambda *ex: dict(zip(['question', 'answer'], ex)))
    return ds


def news_preprocessor(ds):
    def to_inputs_and_targets(ex):
        return {
            'inputs': tf.strings.join(['tajuk: ', ex['question']]),
            'targets': ex['answer'],
        }

    return ds.map(
        to_inputs_and_targets,
        num_parallel_calls = tf.data.experimental.AUTOTUNE,
    )


t5.data.TaskRegistry.remove('news_dataset')
t5.data.TaskRegistry.add(
    'news_dataset',
    dataset_fn = news_dataset,
    splits = ['train'],
    text_preprocessor = [news_preprocessor],
    sentencepiece_model_path = vocab,
    metric_fns = [t5.evaluation.metrics.accuracy],
)

In [5]:
from tqdm import tqdm

@registry.register_problem
class Seq2Seq(text_problems.Text2TextProblem):

    @property
    def approx_vocab_size(self):
        return 32100
    
    @property
    def is_generate_per_split(self):
        return False
    
    @property
    def dataset_splits(self):
        return [{
            "split": problem.DatasetSplit.TRAIN,
            "shards": 100,
        }]
    
    def generate_samples(self, data_dir, tmp_dir, dataset_split):
        del data_dir
        del tmp_dir
        del dataset_split
        
        nq_task = t5.data.TaskRegistry.get("cnn_dataset")
        ds = nq_task.get_dataset(split='qa.tsv', sequence_length={"inputs": 2048, "targets": 1024})
        
        for ex in tqdm(tfds.as_numpy(ds)):
            yield ex
            
        nq_task = t5.data.TaskRegistry.get("multinews_dataset")
        ds = nq_task.get_dataset(split='qa.tsv', sequence_length={"inputs": 2048, "targets": 1024})
        
        for ex in tqdm(tfds.as_numpy(ds)):
            yield ex
            
        nq_task = t5.data.TaskRegistry.get("news_dataset")
        ds = nq_task.get_dataset(split='qa.tsv', sequence_length={"inputs": 2048, "targets": 1024})
        
        for ex in tqdm(tfds.as_numpy(ds)):
            if len(ex['targets']) > 4:
                yield ex
                    
    def generate_encoded_samples(self, data_dir, tmp_dir, dataset_split):
        
        generator = self.generate_samples(data_dir, tmp_dir, dataset_split)
        for sample in generator:
            sample["inputs"] = sample['inputs'].tolist()
            sample["targets"] = sample['targets'].tolist()
            yield sample

In [6]:
!rm -rf t2t-summarization-v2/data

In [7]:
DATA_DIR = os.path.expanduser("t2t-summarization-v2/data")
TMP_DIR = os.path.expanduser("t2t-summarization-v2/tmp")
 
tf.gfile.MakeDirs(DATA_DIR)
tf.gfile.MakeDirs(TMP_DIR)

In [8]:
from tensor2tensor.utils import registry
from tensor2tensor import problems

PROBLEM = 'seq2_seq'
t2t_problem = problems.problem(PROBLEM)
t2t_problem.generate_data(DATA_DIR, TMP_DIR)

0it [00:00, ?it/s]

INFO:tensorflow:Generating case 0.


INFO:tensorflow:Generating case 0.
70196it [02:25, 482.45it/s]
29802it [02:37, 283.19it/s]

INFO:tensorflow:Generating case 100000.


INFO:tensorflow:Generating case 100000.
50787it [04:25, 191.59it/s]
83962it [02:10, 667.16it/s]

...

INFO:tensorflow:Shuffling data...


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


INFO:tensorflow:Data shuffled.


INFO:tensorflow:Data shuffled.


In [9]:
from glob import glob

files = glob('t2t-summarization-v2/data/*')
files

['t2t-summarization-v2/data/seq2_seq-train-00022-of-00100',
 't2t-summarization-v2/data/seq2_seq-train-00017-of-00100',
 't2t-summarization-v2/data/seq2_seq-train-00054-of-00100',
 't2t-summarization-v2/data/seq2_seq-train-00008-of-00100',
 't2t-summarization-v2/data/seq2_seq-train-00014-of-00100',
 't2t-summarization-v2/data/seq2_seq-train-00057-of-00100',
 't2t-summarization-v2/data/seq2_seq-train-00036-of-00100',
 't2t-summarization-v2/data/seq2_seq-train-00038-of-00100',
 't2t-summarization-v2/data/seq2_seq-train-00012-of-00100',
 't2t-summarization-v2/data/seq2_seq-train-00037-of-00100',
 't2t-summarization-v2/data/seq2_seq-train-00053-of-00100',
 't2t-summarization-v2/data/seq2_seq-train-00065-of-00100',
 't2t-summarization-v2/data/seq2_seq-train-00089-of-00100',
 't2t-summarization-v2/data/seq2_seq-train-00046-of-00100',
 't2t-summarization-v2/data/seq2_seq-train-00030-of-00100',
 't2t-summarization-v2/data/seq2_seq-train-00084-of-00100',
 't2t-summarization-v2/data/seq2_seq-tra

In [12]:
for file in files:
    print(file)
    blob = bucket.blob(file)
    blob.upload_from_filename(file)

t2t-summarization-v2/data/seq2_seq-train-00022-of-00100
t2t-summarization-v2/data/seq2_seq-train-00017-of-00100
t2t-summarization-v2/data/seq2_seq-train-00054-of-00100
t2t-summarization-v2/data/seq2_seq-train-00008-of-00100
t2t-summarization-v2/data/seq2_seq-train-00014-of-00100
t2t-summarization-v2/data/seq2_seq-train-00057-of-00100
t2t-summarization-v2/data/seq2_seq-train-00036-of-00100
t2t-summarization-v2/data/seq2_seq-train-00038-of-00100
t2t-summarization-v2/data/seq2_seq-train-00012-of-00100
t2t-summarization-v2/data/seq2_seq-train-00037-of-00100
t2t-summarization-v2/data/seq2_seq-train-00053-of-00100
t2t-summarization-v2/data/seq2_seq-train-00065-of-00100
t2t-summarization-v2/data/seq2_seq-train-00089-of-00100
t2t-summarization-v2/data/seq2_seq-train-00046-of-00100
t2t-summarization-v2/data/seq2_seq-train-00030-of-00100
t2t-summarization-v2/data/seq2_seq-train-00084-of-00100
t2t-summarization-v2/data/seq2_seq-train-00074-of-00100
t2t-summarization-v2/data/seq2_seq-train-00049-o

In [13]:
!rm -rf t2t-summarization-v2 t5-data translated-multinews-*.json cnn-news-translated-*.json translated-*.json